In [ ]:
from __future__ import annotations
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import matplotlib.pyplot as plt

#from coin_ai.alignment.data import recursive_parse_homography_csv

In [ ]:
from coin_ai.alignment.data import HPairDataset, HPathPair, HomographyBatch, AugmentationBuilder

In [ ]:
# from functools import partial
# from typing import NamedTuple
# #from coin_ai.alignment.data import HPair
# from einops import rearrange, repeat
# import torch
# import kornia.geometry as KG
# from torch import Tensor

# class HomographyBatch(NamedTuple):
#     images: Tensor # [2, B, C, H, W]
#     H_12: Tensor # [B, 3, 3]

#     def __repr__(self) -> str:
#         return f'HomographyBatch(images={tuple(self.images.shape)}, H_12={tuple(self.H_12.shape)})'

#     @staticmethod
#     def from_path_pairs(pairs: list[HPathPair]) -> HomographyBatch:
#         images = [torch.stack([p.get_image1(), p.get_image2()], dim=0) for p in pairs]
#         H_12s = [pair.H_12 for pair in pairs]

#         images = torch.stack(images, dim=1)
#         images = rearrange(images, 't b h w c -> t b c h w')
#         H_12s = torch.stack(H_12s)

#         return HomographyBatch(images=images, H_12=H_12s)
    
#     def apply_h(self, H_12: Tensor) -> HomographyBatch:
#         H_12 = H_12.to(self.images.device)
#         images = KG.homography_warp(self.images, H_12)
#         return HomographyBatch(images=images, H_12=self.H_12)
    
#     def slice(self, start: int | None, end: int | None) -> HomographyBatch:
#         images = self.images[:, start:end]
#         H_12 = self.H_12[start:end]

#         return HomographyBatch(images=images, H_12=H_12)
    
#     @property
#     def B(self) -> int:
#         return self.H_12.shape[0]

#     @property
#     def corners(self) -> Tensor:
#         h, w = self.images.shape[-2:]

#         return torch.tensor([
#             [0, 0],
#             [w, 0],
#             [w, h],
#             [0, h]
#         ], dtype=torch.float32)
    
#     def get_alignment_transform(self) -> Tensor:
#         return torch.stack([
#             self.H_12,
#             repeat(torch.eye(3), 'i j -> b i j', b=self.B),
#         ])
    
#     def build_augmentation(self) -> AugmentationBuilder:
#         return AugmentationBuilder(
#             batch=self,
#             transform=repeat(torch.eye(3), 'i j -> c b i j', c=2, b=self.B),
#             target_size=self.images.shape[-2:]
#         )
    
# class AugmentationBuilder(NamedTuple):
#     batch: HomographyBatch
#     transform: Tensor # [2, B, 3, 3]
#     target_size: tuple[int, int]

#     def resize(self, target_size: tuple[int, int]) -> AugmentationBuilder:
#         h, w = target_size
#         resize_transform = KG.get_perspective_transform(
#             self.batch.corners.unsqueeze(0),
#             torch.tensor([
#                 [0, 0],
#                 [w, 0],
#                 [w, h],
#                 [0, h]
#             ], dtype=torch.float32).unsqueeze(0),
#         )
#         resize_transform = repeat(resize_transform, 'B i j -> C B i j', C=2)

#         return AugmentationBuilder(
#             batch=self.batch,
#             transform=resize_transform @ self.transform,
#             target_size=target_size
#         )
    
#     def get_random_homography_transform(self, scale: float = 0.125) -> Tensor:
#         corners = repeat(self.batch.corners, 'i j -> b i j', b=self.batch.B)

#         scale_px = torch.tensor([self.target_size]).reshape(1, 1, 2) * scale
#         corners_1 = corners + torch.randn_like(corners) * scale_px
#         corners_2 = corners + torch.randn_like(corners) * scale_px
#         transform_1 = KG.get_perspective_transform(corners_1, corners_2)
#         transform_2 = KG.get_perspective_transform(corners_2, corners_1)

#         return torch.stack([transform_2, transform_1], dim=0)
    
#     def apply(self, transform: Tensor) -> AugmentationBuilder:
#         assert transform.shape == self.transform.shape
#         t_1, t_2 = transform
#         new_H_12 = t_2 @ self.batch.H_12 @ torch.inverse(t_1)
#         return AugmentationBuilder(
#             batch=self.batch._replace(H_12=new_H_12),
#             transform=transform @ self.transform,
#             target_size=self.target_size
#         )
    
#     def build(self) -> HomographyBatch:
#         warp_fn = partial(KG.warp_perspective, dsize=self.target_size, padding_mode="border")
#         return HomographyBatch(
#             images=torch.vmap(warp_fn, in_dims=(0, 0))(self.batch.images, self.transform),
#             H_12=self.batch.H_12
#         )

In [ ]:
import numpy as np

def draw_corners(homography_batch: HomographyBatch):
    assert homography_batch.B == 1

    fig, (a1, a2) = plt.subplots(1, 2)
    im1, im2 = homography_batch.images.squeeze(1).permute(0, 2, 3, 1).cpu().numpy()
    a1.imshow(im1)
    a2.imshow(im2)

    corners_base = homography_batch.corners.squeeze(0).cpu().numpy()
    corners_base = corners_base / 2 + 128
    corners_base = np.concatenate([corners_base, np.ones((4, 1))], axis=1)
    corners_warp = corners_base @ homography_batch.H_12.squeeze(0).cpu().numpy().T

    corners_warp /= corners_warp[:, 2:]
    a2.plot(corners_warp[:, 0], corners_warp[:, 1], 'ro')
    a1.plot(corners_base[:, 0], corners_base[:, 1], 'ro')

    return fig

In [ ]:
base_path_pairs = HPairDataset.parse_homography_csv('/Users/jatentaki/Data/archeo/coins/krzywousty-homographies/Krzywousty/Typ 2/Awers - stempel a.III.01/homographies.csv')
inferred_path_pairs = HPairDataset.infer_homographies(base_path_pairs)

In [ ]:
len(base_path_pairs), len(inferred_path_pairs)

In [ ]:
batch = HomographyBatch.from_path_pairs(inferred_path_pairs)

In [ ]:
#resized = batch.build_augmentation().resize((256, 256)).build()
builder = batch.build_augmentation()
with_aug = builder.apply(batch.get_alignment_transform()).apply(builder.get_random_homography_transform(scale=0.05)).build()

In [ ]:
for i in range(with_aug.B):
    fig = draw_corners(with_aug.slice(i, i + 1))
    fig.suptitle(f"{i}")

plt.show()

In [ ]:
len(base_path_pairs)